## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cape Town,ZA,-33.93,18.42,59.00,54,0,13.87,clear sky
1,1,Rjukan,NO,59.88,8.59,52.00,99,100,13.00,moderate rain
2,2,Yellowknife,CA,62.46,-114.35,23.00,92,90,9.17,light snow
3,3,Itoman,JP,26.12,127.67,82.40,48,20,14.99,few clouds
4,4,Busselton,AU,-33.65,115.33,60.01,84,100,7.09,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Itoman,JP,26.12,127.67,82.40,48,20,14.99,few clouds
6,6,Butaritari,KI,3.07,172.79,81.70,76,100,12.84,overcast clouds
7,7,Avarua,CK,-21.21,-159.78,75.20,83,100,10.29,light rain
12,12,Airai,TL,-8.93,125.41,79.30,37,0,2.77,clear sky
14,14,Makakilo City,US,21.35,-158.09,84.20,51,90,6.93,overcast clouds
17,17,Atuona,PF,-9.80,-139.03,78.17,77,7,17.78,light rain
20,20,Los Llanos De Aridane,ES,28.66,-17.92,75.20,47,0,11.41,clear sky
21,21,Hithadhoo,MV,-0.60,73.08,83.88,71,100,9.91,light rain
33,33,Fort Wellington,GY,6.40,-57.60,81.72,83,13,13.85,light rain
40,40,Georgetown,MY,5.41,100.34,84.20,78,20,3.36,shower rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             72
City                   72
Country                72
Lat                    72
Lng                    72
Max Temp               72
Humidity               72
Cloudiness             72
Wind Speed             72
Current Description    72
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna('index')

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel_Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel_Name
3,Itoman,JP,82.40,few clouds,26.12,127.67,
6,Butaritari,KI,81.70,overcast clouds,3.07,172.79,
7,Avarua,CK,75.20,light rain,-21.21,-159.78,
12,Airai,TL,79.30,clear sky,-8.93,125.41,
14,Makakilo City,US,84.20,overcast clouds,21.35,-158.09,
17,Atuona,PF,78.17,light rain,-9.80,-139.03,
20,Los Llanos De Aridane,ES,75.20,clear sky,28.66,-17.92,
21,Hithadhoo,MV,83.88,light rain,-0.60,73.08,
33,Fort Wellington,GY,81.72,light rain,6.40,-57.60,
40,Georgetown,MY,84.20,shower rain,5.41,100.34,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # get latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel_Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("hotel not found... skipping.")
        

hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna('index')
clean_hotel_df.count()

City                   72
Country                72
Max Temp               72
Current Description    72
Lat                    72
Lng                    72
Hotel_Name             72
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
#locations = hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))